- Goal

In this assignment, you will write a program that scapes the Wikipedia list of top-level domains and writes a CSV file that contains all the top-level domains and whether their "example" second-level domain resolves to an address that returns an HTTP responses.

- Step 1

Use the Requests module to download the page with the list of top-level domains.

- Step 2

Use BeautifulSoup, regular expressions, or some other means to extract the relevant domains.

- Step 3

Use the Requests package to check whether the "example" second-level domain is valid for each of the top-level domains you extracted.

- Step 4

Write a CSV file (https://docs.python.org/3/library/csv.html (Links to an external site.)) with the results.

In [5]:
#this lib is used for backup, becuase we have to combine requests and bs4 used together to initial web scrapy
import requests
import bs4
#we can just use scrapy to come out same profermacne compared to use to different lab
import scrapy
#Normally, we use scrapy as a bypass program, but we still can use CrawerProcess or CrawerRunner to pipeline scrapy
from scrapy.crawler import CrawlerRunner
from twisted.internet import reactor
from scrapy.utils.log import configure_logging
#If i don't use BS4 to parse html, i can use Selector to extract text content
from scrapy import Selector
#To make extract more specific, we can use re to loacte top level domain phrase
import re
#Finally i didn't use this lib, because it add slash'/'' between two string phrase
from urllib.parse import urljoin
import csv

- this original top domain xpath <code>/html/body/div[1]/div/main/div[3]/div[1]/div/section[3]/table/tbody/tr[2]/td[1]/a</code>
- this Infrusture top level domain <code>/html/body/div[1]/div/main/div[3]/div[1]/div/section[4]/table/tbody/tr[2]/td[1]/a</code>
- find <code>table class=wikitable</code> 
- All of the Top-Level domain have a <code>table</code> element. On this part, wikw website wirte html standard. But 1/3 top domain html resource seems edit later. Theoretically, text should be under <code>/tbody/tr/td/a/text()</code>, but some of them only put text under <code>/td//text()</code> without any indication. So I have to extract twice and emerge two result list.
- actually some mising is not 100% our code problem, it's the nonstandard of this wiki page. For example, every table that represent top level domain(class = wikitable) should have a <code>tr [@vialgn="top"]</code>. Clearly this page is a multi-edit version. Some palces we see this standard code, but some place <code>title</code> not even under <code>td</code> element

In [8]:
#First we input our target URL here
urls_3= ['https://en.m.wikipedia.org/wiki/List_of_Internet_top-level_domains']
#use request to get response
res_3=requests.get(urls_3[0])
#response is a combination, we should use BS4 or Selector to extract text
sel_3 = Selector(text=res_3.content)
soup_3=bs4.BeautifulSoup(res_3.content,'lxml')
#This step is to loacate the wikitable, all of the top level result is uner this class table
wiki_table= sel_3.xpath('.//table[contains(@class,"wikitable")]/tbody/tr')



#Due to the html structure problem, i have to split extraction into two part, and merge them when finished.
domain_key_1=[]
a=0
b=0
#wiki_table is a list, it contain 43 tables who have class=wikitable. We have to iterate all further generations
for i in range(0,len(wiki_table)):
    #We don't extract herf and title, we only what text before</a> becuase some title result is uncompleted
    domain_title_1=(wiki_table[i].xpath('.//td/a//text()'))
    #after xpath, we get the result. But only extract can get the full text result and due to multi-td elements we only extract first
    str_domain_1=domain_title_1.extract_first()
    #str_domain_1 is a none structure result, it have string and None types, we only want string 
    if isinstance(str_domain_1,str)==True:
        #this regex is used to extract  .com result but include non-alphabet letter domain and delete brackets within it
        pattern_1 = re.compile(r'^[\.]\S*')  
        #add the regex result into a list 
        domain_key_1 +=  pattern_1.findall(str_domain_1)
        #this is only for count
        b= b +1
    else:
        a = a +1

domain_key_2=[];c=0;d=0        
for i in range(0,len(wiki_table)):
    domain_title_2=(wiki_table[i].xpath('.//td//text()'))
    str_domain_2=domain_title_2.extract_first()
    if isinstance(str_domain_2,str)==True:
        #print(str_domain)
        pattern_1 = re.compile(r'^[\.]\S*')  
        domain_key_2 +=  pattern_1.findall(str_domain_2)
        c= c +1
    else:
        d = d +1      
        
#we should merge the two list and delete dupulicate    
domain_list = domain_key_1 + list(set(domain_key_2) - set(domain_key_1))


#combine example and top-domain together to get a new fullhttp list url_list
url_list=[]
for key in domain_list:
    url = 'http://example'+ key
    print(url)
    url_list += [url]


file_path='18_Assignment 02_list.csv'        
#this is for test, everytiem running 1000+ requets is to waste time        
url_20=url_list[254:260]    
#Because we have unusual letter in top domain value, we have to encode them into one standard
with open(file_path, 'w',encoding='utf-8-sig',newline='') as csvfile_2:
    writer = csv.writer(csvfile_2)
    #Becuase some url is unexist, so we have to use try function to output and if url is not exist we manully add return code
    for url in url_list:     
        try:
            response = requests.get(url)
            code = response.status_code
            #In this we, we make two column and one row at a time 
            writer.writerow ([url,code])
        except requests.exceptions.ConnectionError:
            code = "Connection refused"
            writer.writerow ([url,code])





http://example.com
http://example.org
http://example.net
http://example.int
http://example.edu
http://example.gov
http://example.mil
http://example.arpa
http://example.ac
http://example.ad
http://example.ae
http://example.af
http://example.ag
http://example.ai
http://example.al
http://example.am
http://example.ao
http://example.aq
http://example.ar
http://example.as
http://example.at
http://example.au
http://example.aw
http://example.ax
http://example.az
http://example.ba
http://example.bb
http://example.bd
http://example.be
http://example.bf
http://example.bg
http://example.bh
http://example.bi
http://example.bj
http://example.bm
http://example.bn
http://example.bo
http://example.br
http://example.bs
http://example.bt
http://example.bw
http://example.by
http://example.bz
http://example.ca
http://example.cc
http://example.cd
http://example.cf
http://example.cg
http://example.ch
http://example.ci
http://example.ck
http://example.cl
http://example.cm
http://example.cn
http://example.co
h

In [151]:
#This part is test part, not important

content = ['.net','.cn','.com','.te','.test']
list_1=[]
for i in range(0,len(content)):
    x1=re.findall('^[\.]\S*', content[i])
    print(x1)
    print(type(x1))
    list_1.append(x1)
print(list_1)


"""


status_code=[]
for key in domain_20: 
    url='http://example'+ key
    print(url)
    try:
        response = requests.get(url)
        code = response.status_code
    except requests.exceptions.ConnectionError:
        code = "Connection refused"
        status_code += str(code)

    
    
    

with open(file_path, 'w',encoding='utf-8',newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(domain_key)
    writer.writerow(status_code)
"""

#Actully we have a lot of unstandard html part (title not in <a> but just in <td>) if i want to improve, i should start another process
#domain_title_2=(wiki_table[i].xpath('.//td//text()'))        
#with


    
#domain_title = wiki_table.xpath('./tbody/tr/td/a/@title')
#domain_key

#print(type(wiki_table))
#print(type(domain_title[9]))

"""
for i in range(0,len(domain_title)):
    domain_key=re.findall("", domain_title[i].extract())
print(domain_key)
"""

['.net']
<class 'list'>
['.cn']
<class 'list'>
['.com']
<class 'list'>
['.te']
<class 'list'>
['.test']
<class 'list'>
[['.net'], ['.cn'], ['.com'], ['.te'], ['.test']]


In [116]:
#Creat spider class
class Wiki_Spider(scrapy.Spider):
    name = 'wiki_spider'
    
    
    def start_request(self):
        urls='https://en.m.wikipedia.org/wiki/List_of_Internet_top-level_domains'
        yield scrapy.Request(url=urls, callback=self.parse)
        
    def parse(self, response):
        #narrow all table which class contain 'wikitable'
        wiki_table = response.xpath('.//table/text()')
        
        #get all element in a and title is the top level domain
        #domain_title = wiki_table.xpath('./a/@title//text()')
        #Extract text
        #domain_text = domin_title.extract_firts()
        print(wiki_table.extract())
        yield wiki_table
 

"""
class Wiki_Requests:
    name='wiki_requests'
    urls= ['https://en.m.wikipedia.org/wiki/List_of_Internet_top-level_domains']
    
    def requests(self):
        response = requests.get(urls[0])
        
configure_logging = ({ 'LOG_FORMAT':'%(levelname)s: %(message)s'})

runner = CrawlerRunner()

d = runner.crawl(Wiki_Spider)

d.addBoth(lambda _: reactor.stop())

reactor.run()
"""

True